# Churn Prediction

In [2]:
from IPython.display import Image
Image(url="./churn_pred.png", height=500)

It's a binary classification, will it churn or will it stays.
It will be a score between 0-1
Likelihood of churn

We should always have some data every month from people leaving the company.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [5]:
# We use the transpose here, to see columns as rows, and be able to see all the values of the columns
# in a single column (easier to see)
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [6]:
# cleaning up some data
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [7]:
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.5,no
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,yes
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,no
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,yes


In [8]:
# if we cannot convert the number we can coerce it with this specific parameter
# total charges mightve been an empty space = " ", when we 
# clened up our data, we converted some of those " " into "_", lets fix that
tc = pd.to_numeric(df.totalcharges, errors='coerce')
df[tc.isnull()][['customerid', 'totalcharges']]

,customerid,totalcharges
488,4472-lvygi,_
753,3115-czmzd,_
936,5709-lvoeq,_
1082,4367-nuyao,_
1340,1371-dwpaz,_
3331,7644-omvmy,_
3826,3213-vvolg,_
4380,2520-sgtta,_
5218,2923-arzlg,_
6670,4075-wkniu,_


In [9]:
df.totalcharges = tc
df.totalcharges.isnull().sum()

11

In [10]:
df.totalcharges = df.totalcharges.fillna(0)

In [11]:
df.totalcharges.isnull().sum()

0

In [12]:
df.churn.head()

0     no
1     no
2    yes
3     no
4    yes
Name: churn, dtype: object

In [13]:
df.churn = (df.churn == "yes").astype(int)

In [14]:
df.churn.head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int64

Setting up the validation framework

60/20/20 distribution
60 for training, 20 for validation, and 20 for test

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
#pull up doc
train_test_split?

In [17]:
# 80/20 split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [18]:
len(df_full_train), len(df_test)

(5634, 1409)

In [19]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [20]:
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [21]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [22]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [23]:
del df_train["churn"]
del df_val["churn"]
del df_test["churn"]

# EDA (Exploratory Data Analysis)

- Check missing values
- look at the target variable
- look at numerical and categorical values

In [24]:
# from the split, the indices get randomized, let's make them look pretty, just bc we can
df_full_train = df_full_train.reset_index(drop=True)

In [25]:
# lets see how many occurrences of each value are for this variable, we can normalize it to a 0-1 split
# the 1's are what is effectively called the churn rate.
df_full_train.churn.value_counts(normalize=True)

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [26]:
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate, 2)

0.27

the reason the above code works, is because mean is solved by taking the sum of items, and dividing them by the number of items in the list
`[1, 0, 1, 0, 0, 0]` is effectively `(1+1) / 6`

In [27]:
numerical = ["tenure", "monthlycharges", "totalcharges"]

In [28]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [29]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod' ]

In [30]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

# Feature Importance

In [31]:
df_full_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
1,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
2,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
3,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
4,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [32]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

0.27682403433476394

In [33]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

0.2632135306553911

In [34]:
global_churn_rate

0.26996805111821087

In [35]:
df_full_train.partner.value_counts()

partner
no     2932
yes    2702
Name: count, dtype: int64

In [36]:
partner_yes = df_full_train[df_full_train.partner == 'yes'].churn.mean()
partner_yes

0.20503330866025166

In [37]:
partner_no = df_full_train[df_full_train.partner == 'no'].churn.mean()
partner_no

0.3298090040927694

Feature Important

- Difference:
    - Global - group > 0 less likely
    - Global - group < 0 more likely

In [38]:
partner_no / global_churn_rate 

1.2216593879412643

In [39]:
partner_yes / global_churn_rate

0.7594724924338315

Risk Ratio

Risk = Group / global churn rate

`> 1: more likely to churn`
`< 1: less likely to churn`

In [40]:
from IPython.display import Image
Image(url="./churn_rate.png", height=500)

```
SELECT
    gender,
    AVG(churn),
    AVG(churn) - global_churn AS diff,
    AVG(churn) / global_churn AS risk
FROM 
    data
GROUP BY
    gender;
```

In [41]:
from IPython.display import display

In [45]:

# Lets us see the relationship between the features, and impact on the churn rate. 
for c in categorical:
    # We are grouping by category
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    # adding the diff
    df_group['diff'] = df_group['mean'] - global_churn_rate
    # also adding the risk
    df_group['risk'] = df_group['mean'] / global_churn_rate
    # new df
    display(df_group)
    print()

,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208


,mean,count,diff,risk
partner,,,,
no,0.329809,2932,0.059841,1.221659
yes,0.205033,2702,-0.064935,0.759472


,mean,count,diff,risk
dependents,,,,
no,0.313760,3968,0.043792,1.162212
yes,0.165666,1666,-0.104302,0.613651


,mean,count,diff,risk
phoneservice,,,,
no,0.241316,547,-0.028652,0.893870
yes,0.273049,5087,0.003081,1.011412


...

,mean,count,diff,risk
streamingtv,,,,
no,0.342832,2246,0.072864,1.269897
no_internet_service,0.077805,1221,-0.192163,0.288201
yes,0.302723,2167,0.032755,1.121328


,mean,count,diff,risk
streamingmovies,,,,
no,0.338906,2213,0.068938,1.255358
no_internet_service,0.077805,1221,-0.192163,0.288201
yes,0.307273,2200,0.037305,1.138182


,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


# Feature Importance: Mutual Information
How much do we know of `churn` by looking at something like `contract`

In [47]:
from sklearn.metrics import mutual_info_score

# This is extremely important because this tells us how much one variable impacts another one
# in this case the churn rate.
mutual_info_score(df_full_train.churn, df_full_train.contract)

0.0983203874041556

In [48]:
mutual_info_score(df_full_train.contract, df_full_train.churn)

0.0983203874041556

In [49]:
mutual_info_score(df_full_train.gender, df_full_train.churn)

0.0001174846211139946

In [50]:
mutual_info_score(df_full_train.partner, df_full_train.churn)

0.009967689095399745

In [51]:
mutual_info_score(df_full_train.partner, df_full_train.churn)

0.009967689095399745

In [52]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [56]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

# Feature Importance: Correlation

How about numerical columns

(Pearson Correlation Coefficient)

x --- y = r

-1 <= r <= 1

LOW(Rarely):
0.0 -0.2
0.0 0.2

MODERATE(Sometimes):
-0.2 -0.5
0.2 0.5

STRONG(Often/Always):
-0.6 -1.0
0.6 1.0

y E {0,1}

x E |R 

x = TENURE -> 0 - 72

y = CHURN -> {0,1}


Positive would mean: More Tenure -> Higher Churn Rate
Negative would mean: More Tenure -> Lower Churn Rate

In [59]:
df_full_train[numerical].corrwith(df_full_train.churn)

# We can interpret these values such as:
# Tenure: The longer a customer is with us, the less likely they are to churn
# Monthly Charges: The higher the monthly charges are, the more likely they are to churn
# Total Charges: The more people pay, the less likely they are to churn

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

In [63]:
df_full_train[df_full_train['tenure'] <= 2].churn.mean()

0.5953420669577875

In [64]:
df_full_train[(df_full_train.tenure > 2) & (df_full_train.tenure <= 12)].churn.mean()

0.3994413407821229

In [65]:
df_full_train[(df_full_train.tenure > 12)].churn.mean()

0.17634908339788277

In [66]:
from IPython.display import Image
Image(url="./correlation.png", height=500)

In [67]:
df_full_train[df_full_train.monthlycharges <= 20].churn.mean()

0.08795411089866156

In [70]:
df_full_train[(df_full_train.monthlycharges > 20) & (df_full_train.monthlycharges <= 50)].churn.mean()

0.18340943683409436

In [71]:
df_full_train[df_full_train.monthlycharges > 50].churn.mean()

0.32499341585462205

In [73]:
df_full_train[numerical].corrwith(df_full_train.churn).abs()

tenure            0.351885
monthlycharges    0.196805
totalcharges      0.196353
dtype: float64